<a href="https://colab.research.google.com/github/ekaterinaKrupkina/decomposer_notebooks/blob/main/separate_steps/experiments__evaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import os
import subprocess
import json
from google.colab import userdata
import shlex
import pandas as pd
import time
import sys
from datetime import datetime


TODO:


1.   func to generate test cases





## Set up environment

## !!! After installation you'll be asked to reload the runtime. Please, REFUSE

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
REQUIREMENTS_PATH = "/content/drive/MyDrive/HarmBench/requirements_full.txt"

In [4]:
!pip install -r {REQUIREMENTS_PATH}

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.0/147.0 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.9/256.9 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.4/865.4 kB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.8/601.8 kB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.8/29.8 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━

In [5]:
%cd /content/drive/MyDrive/HarmBench

/content/drive/MyDrive/HarmBench


In [7]:
# set environ variables
os.environ["WANDB__EXECUTABLE"] = "/usr/local/bin/python"
os.environ["WANDB_API_KEY"] = userdata.get("WANDB_API_KEY")
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["GITHUB_TOKEN"] = userdata.get("GITHUB_TOKEN")
os.environ["OPENAI_BASE_URL"] = "https://api.openai.com/v1"
os.environ["MISTRAL_API_KEY"] = userdata.get("MISTRAL_API_KEY")
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")


KeyboardInterrupt: 

## Functions

In [ ]:
def run_evaluate_completions(behaviors_path):
    """
    Run the evaluate_completions.py script as a subprocess.

    This function executes the evaluate_completions.py script with the specified parameters,
    captures its output in real-time, and prints it to the console.

    Args:
        behaviors_path (str): Path to the behaviors dataset file.
        completions_path (str): Path to the completions file to be evaluated.
        save_path (str): Path where the evaluation results will be saved.

    Returns:
        None

    Raises:
        subprocess.CalledProcessError: If the subprocess returns a non-zero exit status.
    """

    # Set up variables
    MODEL_TARGET = os.getenv("MODEL_TARGET")
    MODEL_DECOMPOSER = os.getenv("MODEL_DECOMPOSER")

    completions_path = f'/content/drive/MyDrive/HarmBench/results/completions/completions_{MODEL_TARGET}_{MODEL_DECOMPOSER}_full.json'
    save_path = f'/content/drive/MyDrive/HarmBench/results/evaled_{MODEL_TARGET}_{MODEL_DECOMPOSER}_full.json'

    # Execute the script as a subprocess
    command = [
        sys.executable,
        "-u",
        "evaluate_completions.py",
        f"--behaviors_path={behaviors_path}",
        f"--completions_path={completions_path}",
        f"--save_path={save_path}"
    ]

    try:
        process = subprocess.Popen(
            command,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True
        )

        for line in process.stdout:
            print(line, end='')

    except subprocess.CalledProcessError as e:
        print(f"Error: {e}")

In [ ]:
import wandb

def process_results(project="test_db", entity="palisaderesearch"):
    """
    Process the results of the evaluation and log them to Weights & Biases.

    This function reads the evaluation results from a JSON file, creates a DataFrame,
    and logs the results to Weights & Biases for tracking and visualization.

    Args:
        file (str): Name of the JSON file containing the evaluation results (without .json extension).
        decomposer (bool): Flag indicating whether a decomposer was used in the experiment.
        config_file (str): Path to the JSON config file to append to the run's config.

    Returns:
        None
    """
    # Set up variables
    MODEL_TARGET = os.getenv("MODEL_TARGET")
    MODEL_DECOMPOSER = os.getenv("MODEL_DECOMPOSER")
    config_file=f'/content/drive/MyDrive/HarmBench/config_{MODEL_TARGET}_{MODEL_DECOMPOSER}.json'
    evaled_file = f'/content/drive/MyDrive/HarmBench/results/evaled_{MODEL_TARGET}_{MODEL_DECOMPOSER}_full.json'

    # Load the JSON data
    data_eval = pd.read_json(evaled_file)
    data_summ = pd.read_json(f"/content/drive/MyDrive/HarmBench/summary_{MODEL_TARGET}_{MODEL_DECOMPOSER}.json")

    # Convert JSON data to DataFrame
    df_evaled = pd.DataFrame([{"category": key, **value[0]} for key, value in data_eval.items()])
    df = df_evaled.merge(data_summ, on='test_case', how='inner')

    # Add additional columns
    df["target_model"] = os.getenv("MODEL_TARGET")
    df["decomposer_model"] = os.getenv("MODEL_DECOMPOSER")
    df['composer_model'] = os.getenv("MODEL_COMPOSER")
    df["surrogate_model"] = os.getenv("MODEL_SURROGATE")
    df["ASR"] = df["label"].mean()

    # Remove duplicates
    df = df.drop_duplicates(subset=['test_case'])

    # Remove the 'generation_y' column
    df.drop('generation_y', axis=1, inplace=True)

    # Rename 'generation_x' to 'generation'
    df.rename(columns={'generation_x': 'generation'}, inplace=True)

    # Rename columns to start with 'tokens_' or 'price_'
    rename_dict = {
        'surrogate_tokens': 'tokens_surrogate',
        'target_tokens': 'tokens_target',
        'decomposer_tokens': 'tokens_decomposer',
        'composer_tokens': 'tokens_composer',
        'surrogate_price': 'price_surrogate',
        'target_price': 'price_target',
        'decomposer_price': 'price_decomposer',
        'composer_price': 'price_composer'
    }
    df.rename(columns=rename_dict, inplace=True)

    # Print the DataFrame
    print(df.head())

    # Initialize W&B run
    run = wandb.init(
        entity=entity,
        project=project,
        save_code=False,
        name=f"decomp_{MODEL_TARGET}_{MODEL_DECOMPOSER}_full"
    )

    # Load the config file and update run config
    with open(config_file, 'r') as f:
        config_data = json.load(f)
        run.config.update(config_data)

    # Log the DataFrame as a W&B Table
    run.log({"harmbench": wandb.Table(dataframe=df)})
    run.summary["harmbench_mean"] = df["label"].mean()
    run.summary["target_model"] = os.getenv("MODEL_TARGET")

    # Finish the W&B run
    run.finish()


In [ ]:
def experiment(experiment_params):
    """
    Run a complete experiment pipeline including completion generation, evaluation, and result processing.

    This function sets up the environment, runs the necessary scripts for generating completions
    and evaluating them, and then processes and logs the results.

    Args:
        experiment_params (dict): A dictionary containing experiment parameters including model names.

    Returns:
        None
    """

    # Stop the decomposer server if it's running
    !pkill --full 'web\.py'

    # Set up environment variables
    for k, v in experiment_params.items():
        os.environ[k] = str(v)

    os.environ["OPENAI_BASE_URL"] = "https://api.openai.com/v1"
    os.environ["BADLLAMA_ENDPOINT_URL"] = "https://kdko1b8bau1a25bc.eu-west-1.aws.endpoints.huggingface.cloud"

    # Run the decomposer as a background process
    run_in_background("./Decomposition-Harness/server/web.py", "output.log")

    # waiting for decomposer server initialization
    time.sleep(10)

    os.environ["OPENAI_BASE_URL"] = "http://127.0.0.1:8082"

    try:

      run_evaluate_completions(
          behaviors_path="./data/behavior_datasets/harmbench_behaviors_text_all.csv"
      )

      process_results(project='test_db')

    except Exception as e:
      print(e)
      pass

    for k, v in experiment_params.items():
      del os.environ[k]



In [ ]:
def get_summary_table(number_of_runs, project_name, entity_name='palisaderesearch'):
    api = wandb.Api()

    # Get all runs for the specified project
    runs = api.runs(f"{entity_name}/{project_name}")

    # Sort runs by creation time (newest first)
    sorted_runs = sorted(runs, key=lambda run: datetime.strptime(run.created_at, "%Y-%m-%dT%H:%M:%S"), reverse=True)

    # Get the n newest run IDs
    newest_run_ids = [run.id for run in sorted_runs[:number_of_runs]]

    full_table = pd.DataFrame()

    # Iterate over the newest run IDs
    for run_id in newest_run_ids:
        artifact_name = f"run-{run_id}-harmbench:v0"
        artifact_full_name = f"{entity_name}/{project_name}/{artifact_name}"

        try:
            # Attempt to retrieve the artifact
            artifact = api.artifact(artifact_full_name)
            table = artifact.get("harmbench")
            df = pd.DataFrame(data=table.data, columns=table.columns)

            # Aggregate the data
            agg_cols = ['price_total', 'tokens_total', 'ASR']
            group_by_cols = ['target_model', 'decomposer_model']
            aggregated_df = df.groupby(group_by_cols)[agg_cols].mean().reset_index()

            # Rename the columns
            aggregated_df = aggregated_df.rename(columns={
                'price_total': 'AVG_price',
                'tokens_total': 'AVG_tokens'
            })

            # Add run_id column
            aggregated_df['run_id'] = run_id

            full_table = pd.concat([full_table, aggregated_df], ignore_index=True)

        except wandb.errors.CommError:
            print(f"Artifact not found: {artifact_full_name}")

    # Initialize a new wandb run
    with wandb.init(project=project_name, entity=entity_name, job_type="summary_table") as run:
        # Log the summary table
        run.log({"summary_table": wandb.Table(dataframe=full_table)})

    # Print the resulting table
    print("Summary Table:")
    print(full_table)

    return full_table

## Run experiments

In [ ]:
experiments_dict = {
    #gpt-4o as target model
    # "exp1": {
    #     "MODEL_TARGET": "gpt-4o-2024-05-13",
    #     "MODEL_COMPOSER": "gpt-3.5-turbo-1106",
    #     "MODEL_DECOMPOSER": "mistral-medium",
    #     "MODEL_SURROGATE": "gpt-3.5-turbo-1106"
    # },
    # "exp2": {
    #     "MODEL_TARGET": "gpt-4o-2024-05-13",
    #     "MODEL_COMPOSER": "gpt-3.5-turbo-1106",
    #     "MODEL_DECOMPOSER": "gpt-3.5-turbo-1106",
    #     "MODEL_SURROGATE": "gpt-3.5-turbo-1106"
    # },
    # "exp3": {
    #     "MODEL_TARGET": "gpt-4o-2024-05-13",
    #     "MODEL_COMPOSER": "badllama",
    #     "MODEL_DECOMPOSER": "badllama",
    #     "MODEL_SURROGATE": "gpt-3.5-turbo-1106"
    # },
    # #gpt-3.5-turbo-1106 as target model
    # "exp4": {
    #     "MODEL_TARGET": "gpt-3.5-turbo-1106",
    #     "MODEL_COMPOSER": "gpt-3.5-turbo-1106",
    #     "MODEL_DECOMPOSER": "Mistral 7B",
    #     "MODEL_SURROGATE": "gpt-3.5-turbo-1106"
    # },
    # "exp5": {
    #     "MODEL_TARGET": "gpt-3.5-turbo-1106",
    #     "MODEL_COMPOSER": "gpt-3.5-turbo-1106",
    #     "MODEL_DECOMPOSER": "gpt-3.5-turbo-1106",
    #     "MODEL_SURROGATE": "gpt-3.5-turbo-1106"
    # },
    # "exp6": {
    #     "MODEL_TARGET": "gpt-3.5-turbo-1106",
    #     "MODEL_COMPOSER": "badllama",
    #     "MODEL_DECOMPOSER": "badllama",
    #     "MODEL_SURROGATE": "gpt-3.5-turbo-1106"
    # },
    # # gpt-4o as target model, no decomposer
    "exp7": {
        "MODEL_TARGET": "gpt-4o-2024-05-13",
        "MODEL_DECOMPOSER": "",
        "MAX_QUERY_RETRIES_TARGET": 1
    },
    # gpt-3.5-turbo-1106 as target model, no decomposer
    "exp8": {
        "MODEL_TARGET": "gpt-3.5-turbo-1106",
        "MODEL_DECOMPOSER": "",
        "MAX_QUERY_RETRIES_TARGET": 1
    }
}

# Print the updated dictionary to verify
for exp, params in experiments_dict.items():
    print(f"{exp}: {params}")


exp7: {'MODEL_TARGET': 'gpt-4o-2024-05-13', 'MODEL_DECOMPOSER': '', 'MAX_QUERY_RETRIES_TARGET': 1}
exp8: {'MODEL_TARGET': 'gpt-3.5-turbo-1106', 'MODEL_DECOMPOSER': '', 'MAX_QUERY_RETRIES_TARGET': 1}


In [ ]:
for experiment_name, experiment_params in experiments_dict.items():
  print(experiment_name)
  experiment(experiment_params)
  print('======================================')
print("===================END=======================")

exp7
Namespace(cls_path='cais/HarmBench-Llama-2-13b-cls', behaviors_path='./data/behavior_datasets/harmbench_behaviors_text_all.csv', completions_path='./results/completions/completions_gpt-4o-2024-05-13__full.json', save_path='./results/evaled_gpt-4o-2024-05-13__full.json', include_advbench_metric=False, num_tokens=512)
2024-07-16 06:57:15.054158: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-16 06:57:15.096904: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-16 06:57:15.096948: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT

harmbench_mean,0.0
target_model,gpt-4o-2024-05-13


wandb: WARNING Saving files without folders. If you want to preserve subdirectories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")


harmbench_mean,0.0
target_model,gpt-4o-2024-05-13


              Metric          Avg
0   surrogate_tokens  1153.000000
1      target_tokens  2612.000000
2  decomposer_tokens   902.000000
3    composer_tokens  3336.000000
4    surrogate_price     0.002191
5       target_price     0.037980
6   decomposer_price          NaN
7     composer_price          NaN
8       tokens_total  8003.000000
9        price_total     0.040171
exp8
Namespace(cls_path='cais/HarmBench-Llama-2-13b-cls', behaviors_path='./data/behavior_datasets/harmbench_behaviors_text_all.csv', completions_path='./results/completions/completions_gpt-3.5-turbo-1106__full.json', save_path='./results/evaled_gpt-3.5-turbo-1106__full.json', include_advbench_metric=False, num_tokens=512)
2024-07-16 06:58:38.900221: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07

harmbench_mean,0.0
target_model,gpt-3.5-turbo-1106


              Metric          Avg
0   surrogate_tokens  1153.000000
1      target_tokens  2612.000000
2  decomposer_tokens   902.000000
3    composer_tokens  3336.000000
4    surrogate_price     0.002191
5       target_price     0.037980
6   decomposer_price          NaN
7     composer_price          NaN
8       tokens_total  8003.000000
9        price_total     0.040171
===================END=======================


In [ ]:
# import gc
# import torch

# gc.collect()

# torch.cuda.empty_cache()

## Push summary table to Wandb



In [ ]:
get_summary_table(8, 'test_db')

Artifact not found: palisaderesearch/test_db/run-sq1n843t-harmbench:v0


wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  


Summary Table:
         target_model    decomposer_model  AVG_price  AVG_tokens       ASR  \
0  gpt-3.5-turbo-1106                       0.000563     369.150  0.550000   
1   gpt-4o-2024-05-13                       0.005330     471.575  0.350000   
2  gpt-3.5-turbo-1106            badllama   0.008170    9067.600  0.833333   
3  gpt-3.5-turbo-1106  gpt-3.5-turbo-1106   0.012563    9191.600  0.350000   
4  gpt-3.5-turbo-1106          Mistral 7B   0.013951   11397.400  0.375000   
5   gpt-4o-2024-05-13            badllama   0.056761   11560.750  0.784314   
6   gpt-4o-2024-05-13  gpt-3.5-turbo-1106   0.142520   36179.375  0.311111   

     run_id  
0  ob96qqv2  
1  nbsi1u4h  
2  quci9j69  
3  k666pmo8  
4  dbarqso5  
5  nc965a5r  
6  y4y07wiq  


,target_model,decomposer_model,AVG_price,AVG_tokens,ASR,run_id
0,gpt-3.5-turbo-1106,,0.000563,369.150,0.550000,ob96qqv2
1,gpt-4o-2024-05-13,,0.005330,471.575,0.350000,nbsi1u4h
2,gpt-3.5-turbo-1106,badllama,0.008170,9067.600,0.833333,quci9j69
3,gpt-3.5-turbo-1106,gpt-3.5-turbo-1106,0.012563,9191.600,0.350000,k666pmo8
4,gpt-3.5-turbo-1106,Mistral 7B,0.013951,11397.400,0.375000,dbarqso5
5,gpt-4o-2024-05-13,badllama,0.056761,11560.750,0.784314,nc965a5r
6,gpt-4o-2024-05-13,gpt-3.5-turbo-1106,0.142520,36179.375,0.311111,y4y07wiq




---

end of file